# TOIs

In [8]:
import sys
# import os
# assert os.environ['CONDA_DEFAULT_ENV']=='ttv'
assert sys.executable=="/ut3/jerome/miniconda3/envs/ttv/bin/python"

In [9]:
import sys
sys.path.insert(0, '../code')
from utils import *

In [10]:
df = get_tois(clobber=True) #set to True only every new TOI releases
df.tail()

Saved:  TOIs with TFPWG disposition==FP are removed.


,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Radius (R_Sun) err,Stellar Metallicity,Stellar Metallicity err,Stellar Mass (M_Sun),Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments
7336,444159424,7159.01,NaN,3,4,3,3,1,4,4,...,NaN,NaN,NaN,NaN,NaN,"16,17,57,77",2024-10-24,2024-10-23,2024-10-28,found in faint-star QLP search
7337,154264903,7160.01,NaN,3,4,3,3,1,4,4,...,0.05,NaN,NaN,NaN,NaN,"16,17,56,57,76,77",2024-10-24,2024-10-23,2024-10-28,found in faint-star QLP search
7338,415461195,7161.01,NaN,3,4,3,3,1,4,4,...,NaN,NaN,NaN,NaN,NaN,"16,17,57,76,77",2024-10-24,2024-10-23,2024-10-28,found in faint-star QLP search
7339,341005781,7162.01,NaN,3,4,3,3,1,4,4,...,0.03,NaN,NaN,0.558261,0.028465,"56,57,76,77",2024-10-24,2024-10-23,2024-10-28,found in faint-star QLP search
7340,343774169,7163.01,NaN,3,4,3,3,1,4,4,...,0.04,NaN,NaN,0.878000,0.107367,"16,17,57,77",2024-10-24,2024-10-23,2024-10-28,found in faint-star QLP search


In [11]:
tics = df['TIC ID'].unique()
len(tics)

5985

In [12]:
len(df['TOI'].unique())

6258

In [13]:
nsys = df['TIC ID'].value_counts()
nsys

TIC ID
425997655    5
367630162    5
251848941    4
29781292     4
150428135    4
            ..
74274839     1
74881311     1
75744886     1
76663549     1
343774169    1
Name: count, Length: 5985, dtype: int64

In [14]:
tics_multis = list(nsys[nsys>1].index)
print(len(tics_multis))

215


## TOI 1136 (Dai+2023)
First order MMR: 3:2, 2:1, 3:2, 7:5, and 3:2

In [15]:
col = 'Period (days)'
d = df[df['TIC ID']==142276270].sort_values(by=col)
d[col]

1133     6.258606
1132    12.518728
1135    18.795547
1134    26.320876
Name: Period (days), dtype: float64

In [16]:
# add two outer planets
periods = [4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]
periods

[4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]

https://arxiv.org/abs/1208.3312

$$
\frac{p'}{p} \sim \frac{j}{j-1}
$$

proximity to resonance:
$$
\Delta = \frac{p'}{p} \frac{j-1}{j} - 1
$$

In [17]:
get_orbit_pairs(10, order=1)

[(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]

In [18]:
resonances_1st_order, deltas = get_resonant_pairs(periods, order=1, tol=0.01)
resonances_1st_order, deltas

(['3:2 | P=(4.17,6.26) (delta=0.03%)',
  '2:1 | P=(6.26,12.52) (delta=0.04%)',
  '3:2 | P=(12.52,18.80) (delta=0.11%)',
  '3:2 | P=(26.32,39.54) (delta=0.16%)'],
 [0.0003067499364933779,
  0.0003891485876383882,
  0.001072737685682057,
  0.0016314412161837488])

In [19]:
resonances_2nd_order, deltas = get_resonant_pairs(periods, order=2, tol=0.01)
resonances_2nd_order, deltas

(['6:4 | P=(4.17,6.26) (delta=0.03%)',
  '4:2 | P=(6.26,12.52) (delta=0.04%)',
  '6:4 | P=(12.52,18.80) (delta=0.11%)',
  '7:5 | P=(18.80,26.32) (delta=0.01%)',
  '6:4 | P=(26.32,39.54) (delta=0.16%)'],
 [0.0003067499364933779,
  0.0003891485876383882,
  0.001072737685682057,
  0.00010182804131475542,
  0.0016314412161837488])

In [20]:
from tess_stars2px import tess_stars2px_function_entry

ticid = 231663901
data_json = get_tfop_info(f"TIC {ticid}")
ra = float(data_json['coordinates']['ra'])
dec = float(data_json['coordinates']['dec'])
outID, outEclipLong, outEclipLat, outSec, outCam, outCcd, \
            outColPix, outRowPix, scinfo = tess_stars2px_function_entry(
                    ticid, ra, dec)
outSec

array([ 1, 27, 67])

## All TOIs

* Science: Discovery of new TTV systems

In [36]:
from tqdm.notebook import tqdm

ttv_1st_order = []       # commensurability in 1 planet pair 
ttv_2nd_order = []
ttv_chain = []           # more than 1 planet pair
deltas1 = []
deltas2 = []
texts = []
sectors = []
for tic in tqdm(tics_multis):
    dd = df[df['TIC ID']==tic].squeeze()
    # make sure depth>1ppt
    if dd['Depth (ppm)'].any():
        if (dd['Depth (ppm)']>1_000).all():
            
            periods = dd["Period (days)"].values
            Nplanets = len(periods)
            toi = str(dd.iloc[0].TOI).split('.')[0]
            
            resonances_1st_order, delta1 = get_resonant_pairs(periods, order=1, tol=0.01);
            resonances_2nd_order, delta2 = get_resonant_pairs(periods, order=2, tol=0.01);
            msg = f"TOI-{str(toi).zfill(4)} | [TIC-{tic}](https://exofop.ipac.caltech.edu/tess/target.php?id={tic}) (nplanets={Nplanets}):<br>\n"
            show = False
            if len(resonances_1st_order)>=1:
                msg += f"First Order: {' '.join(resonances_1st_order)}<br>\n"
                ttv_1st_order.append(tic)
                deltas1.append(min(delta1))
                if len(resonances_1st_order)>=2:
                    ttv_chain.append(tic)         
                show = True
            if len(resonances_2nd_order)>=1:
                if len(resonances_1st_order)>=1:
                    i = int(resonances_1st_order[0].split(':')[0])
                    j = int(resonances_2nd_order[0].split(':')[0])
                    if j/i!=2:
                        msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
                        ttv_2nd_order.append(tic)
                        deltas2.append(min(delta2))
                        show = True
                else:
                    msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
                    ttv_2nd_order.append(tic)
                    deltas2.append(min(delta2))
                    show = True
        
            # check if observed by TESS
            if tic is not None:
                data_json = get_tfop_info(f"TIC {tic}")
                ra = float(data_json['coordinates']['ra'])
                dec = float(data_json['coordinates']['dec'])
                outID, outEclipLong, outEclipLat, outSec, outCam, outCcd, \
                            outColPix, outRowPix, scinfo = tess_stars2px_function_entry(
                                    tic, ra, dec)
                if len(outSec)>0:
                    msg += f"TESS sectors={outSec}<br>\n"
                    
            if show:
                print(msg)
                texts.append(msg)

  0%|          | 0/215 [00:00<?, ?it/s]

TOI-0663 | [TIC-54962195](https://exofop.ipac.caltech.edu/tess/target.php?id=54962195) (nplanets=3):<br>
First Order: 3:2 | P=(4.70,7.10) (delta=0.76%)<br>
TESS sectors=[ 9 35 62]<br>

TOI-2267 | [TIC-459837008](https://exofop.ipac.caltech.edu/tess/target.php?id=459837008) (nplanets=3):<br>
First Order: 9:8 | P=(2.03,2.29) (delta=0.02%)<br>
TESS sectors=[19 20 25 26 40 52 53 59 60 73 79]<br>



/mnt_ut3/raid_ut3/home/jerome/github/research/project/young_ttvs/notebooks/../code/utils.py:319: RuntimeWarning: divide by zero encountered in double_scalars
  delta = abs((Pout/Pin)*(i/j) - 1)


TOI-6109 | [TIC-384984325](https://exofop.ipac.caltech.edu/tess/target.php?id=384984325) (nplanets=2):<br>
First Order: 3:2 | P=(5.70,8.54) (delta=0.05%)<br>
TESS sectors=[18 58]<br>

TOI-0406 | [TIC-153065527](https://exofop.ipac.caltech.edu/tess/target.php?id=153065527) (nplanets=2):<br>
First Order: 2:1 | P=(6.61,13.18) (delta=0.41%)<br>
TESS sectors=[ 3  4 30 31]<br>

TOI-6969 | [TIC-17129975](https://exofop.ipac.caltech.edu/tess/target.php?id=17129975) (nplanets=2):<br>
Second Order: 3:1 | P=(1.51,4.56) (delta=0.31%)
TESS sectors=[14 15 41 55 75]<br>

TOI-5812 | [TIC-388909695](https://exofop.ipac.caltech.edu/tess/target.php?id=388909695) (nplanets=2):<br>
First Order: 4:3 | P=(10.15,13.47) (delta=0.44%)<br>
TESS sectors=[55 82]<br>

TOI-4323 | [TIC-143022742](https://exofop.ipac.caltech.edu/tess/target.php?id=143022742) (nplanets=2):<br>
First Order: 3:2 | P=(14.68,21.97) (delta=0.21%)<br>
TESS sectors=[ 4 31]<br>

TOI-6022 | [TIC-455947620](https://exofop.ipac.caltech.edu/tess/t

In [37]:
len(ttv_1st_order), len(ttv_2nd_order)

(9, 1)

In [38]:
import pandas as pd

s1 = pd.DataFrame(np.c_[ttv_1st_order,deltas1], columns=['tic','delta_min'])
s1['1st order'] = True
s1.sort_values(by='delta_min')

,tic,delta_min,1st order
1,459837008.0,0.000158,True
2,384984325.0,0.000488,True
6,455947620.0,0.001917,True
5,143022742.0,0.002051,True
3,153065527.0,0.004089,True
4,388909695.0,0.004407,True
7,374180079.0,0.005803,True
0,54962195.0,0.007626,True
8,55652896.0,0.007826,True


In [39]:
s2 = pd.DataFrame(np.c_[ttv_2nd_order,deltas2], columns=['tic','delta_min'])
s2['2nd order'] = True
s2.sort_values(by='delta_min')

,tic,delta_min,2nd order
0,17129975.0,0.003054,True


In [40]:
res = pd.merge(s1, s2, how='outer')
res = res.replace(np.nan, False)
res['tic'] = res.tic.map(int)
res = res.sort_values(by='delta_min')
res.head()

/tmp/ipykernel_14806/947716635.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res = res.replace(np.nan, False)


,tic,delta_min,1st order,2nd order
9,459837008,0.000158,True,False
6,384984325,0.000488,True,False
8,455947620,0.001917,True,False
3,143022742,0.002051,True,False
0,17129975,0.003054,False,True


In [41]:
len(res)

10

In [42]:
len(texts), len(res)

(10, 10)

## Sort by delta

In [43]:
for tic in res.tic:
    idx = [True if i.split('TIC-')[1].split(']')[0]==str(tic) else False for i in texts]
    x = np.array(texts)[idx]
    print(x[0] if len(x)>0 else '')

TOI-2267 | [TIC-459837008](https://exofop.ipac.caltech.edu/tess/target.php?id=459837008) (nplanets=3):<br>
First Order: 9:8 | P=(2.03,2.29) (delta=0.02%)<br>
TESS sectors=[19 20 25 26 40 52 53 59 60 73 79]<br>

TOI-6109 | [TIC-384984325](https://exofop.ipac.caltech.edu/tess/target.php?id=384984325) (nplanets=2):<br>
First Order: 3:2 | P=(5.70,8.54) (delta=0.05%)<br>
TESS sectors=[18 58]<br>

TOI-6022 | [TIC-455947620](https://exofop.ipac.caltech.edu/tess/target.php?id=455947620) (nplanets=2):<br>
First Order: 5:4 | P=(1.55,1.93) (delta=0.19%)<br>
TESS sectors=[16 17 24 57 78]<br>

TOI-4323 | [TIC-143022742](https://exofop.ipac.caltech.edu/tess/target.php?id=143022742) (nplanets=2):<br>
First Order: 3:2 | P=(14.68,21.97) (delta=0.21%)<br>
TESS sectors=[ 4 31]<br>

TOI-6969 | [TIC-17129975](https://exofop.ipac.caltech.edu/tess/target.php?id=17129975) (nplanets=2):<br>
Second Order: 3:1 | P=(1.51,4.56) (delta=0.31%)
TESS sectors=[14 15 41 55 75]<br>

TOI-0406 | [TIC-153065527](https://exo

In [29]:
res.to_csv('../data/tics_resonance_from_toi.csv')

In [30]:
idx = df['TIC ID'].isin(res.tic)
tois_in_resonance = df['TOI'][idx].apply(lambda x: str(x).split('.')[0].zfill(4)).unique()
tois_in_resonance

array(['0174', '0175', '0216', '0270', '0282', '0396', '0406', '0663',
       '0696', '0700', '0707', '1097', '1136', '1339', '1408', '1446',
       '1670', '1730', '1749', '2076', '2104', '2267', '2269', '4323',
       '4495', '4647', '5493', '5520', '5812', '6022', '6054', '6109',
       '6265', '6276', '6969'], dtype=object)

In [31]:
np.savetxt('../data/tics_resonance_from_toi.txt', res.tic.tolist(), fmt="%d")
np.savetxt('../data/tois_resonance_from_toi.txt', tois_in_resonance, fmt="%s")
np.savetxt('../data/tics_resonance_from_toi_1st_order.txt', ttv_1st_order, fmt="%d")
np.savetxt('../data/tics_resonance_from_toi_2nd_order.txt', ttv_2nd_order, fmt="%d")

## Transit timing database
* https://ui.adsabs.harvard.edu/abs/2022ApJS..259...62I/abstract

In [27]:
#https://github.com/transit-timing/tt/blob/master/3_database/table4.csv
url = "https://raw.githubusercontent.com/transit-timing/tt/master/3_database/table4.csv"
df2 = pd.read_csv(url)
df2

,System,Orbit number,T_mid,Uncertainty (days),Time System,#,Reference
0,CoRoT-01,-1412,2.454138e+06,0.000470,BJD,1,2009A&A...506..369B
1,CoRoT-01,-1411,2.454140e+06,0.000590,BJD,1,2009A&A...506..369B
2,CoRoT-01,-1410,2.454141e+06,0.000620,BJD,1,2009A&A...506..369B
3,CoRoT-01,-1409,2.454143e+06,0.000390,BJD,1,2009A&A...506..369B
4,CoRoT-01,-1408,2.454144e+06,0.001630,BJD,1,2009A&A...506..369B
...,...,...,...,...,...,...,...
8662,XO-7,214,2.459407e+06,0.000510,BJD_TDB,1,This work
8663,XO-7,215,2.459410e+06,0.000467,BJD_TDB,1,This work
8664,XO-7,216,2.459413e+06,0.000508,BJD_TDB,1,This work
8665,XO-7,217,2.459415e+06,0.000517,BJD_TDB,1,This work
